In [ ]:
import pandas as pd
import glob
import os
import numpy as np
import sys

# --- Configuration ---
# Folder containing all 29 NG-STAR .xlsx files
NGSTAR_INPUT_DIR = "ngstar_outputs/"
# File with your RGI (CARD) results
RGI_INPUT_FILE = "RGI_tidy_filtered.csv"
# Name of the final output file
FINAL_OUTPUT_FILE = "CARD_vs_NGSTAR_FINAL_COMPARISON.csv"


def clean_ngstar_locus(locus_name):
    """
    Cleans locus names from PubMLST (e.g., 'NG_gyrA' -> 'gyrA')
    """
    if pd.isna(locus_name):
        return None
    
    locus_name = str(locus_name).strip().replace("'", "") 
    
    if locus_name == 'NEIS1753':
        return 'penA'
    if 'gyrA' in locus_name:
        return 'gyrA'
    if 'parC' in locus_name:
        return 'parC'
    if 'mtrR' in locus_name:
        return 'mtrR'
    if 'ponA' in locus_name:
        return 'ponA'
    if 'porB' in locus_name:
        return 'porB'
    if 'NG_23S' in locus_name:
        return '23S_rRNA'
    
    return locus_name


def map_aro_to_ngstar_v5(aro_string):
    """
    Creates a reliable mapping to NG-STAR loci based on the gene name from CARD (Best_Hit_ARO).
    This uses our debugged V5 logic.
    """
    if pd.isna(aro_string):
        return None
        
    aro_lower = str(aro_string).lower()
    
    # --- First, the most specific NG-STAR loci ---
    if '23s' in aro_lower and ('a2059g' in aro_lower or 'c2611t' in aro_lower):
        return '23S_rRNA'

    # --- General NG-STAR loci ---
    if 'gyra' in aro_lower:
        return 'gyrA'
    if 'parc' in aro_lower:
        return 'parC'
    if 'mtrr' in aro_lower:
        return 'mtrR'
    if 'pona' in aro_lower:
        return 'ponA'
    if 'porb' in aro_lower or 'porin pib' in aro_lower:
        return 'porB'
    # V5 FIX: penA is also known as PBP2
    if 'pena' in aro_lower or 'pbp2' in aro_lower:
        return 'penA'

    # --- Genes tracked by RGI but NOT NG-STAR ---
    if 'tet(m)' in aro_lower:
        return 'tet(M)'
    if 'blatem' in aro_lower:
        return 'blaTEM'
    if 'oxa' in aro_lower:
        return 'blaOXA'
    if 'rpsj' in aro_lower:
        return 'rpsJ'

    return 'Other'


def check_concordance(row):
    """
    Checks for concordance between NG-STAR and RGI.
    """
    ngstar_found = pd.notna(row['NGSTAR_Allele'])
    rgi_found = pd.notna(row['Best_Hit_ARO'])
    
    if ngstar_found and rgi_found:
        return 'Match (Both Found)'
    elif ngstar_found and not rgi_found:
        return 'NG-STAR Only (RGI Miss)'
    elif not ngstar_found and rgi_found:
        return 'RGI Only (Not in NG-STAR)'
    else:
        return 'N/A (Both Miss)'

def aggregate_strings(series):
    """
    Helper function to collect all unique RGI findings.
    """
    unique_values = series.dropna().unique()
    if len(unique_values) == 0:
        return None
    return '; '.join(unique_values)

def main():
    print(f"--- Script Start: Comparison of CARD (RGI) vs NG-STAR (PubMLST) ---")
    
    # === PART 1: ASSEMBLING THE 'GOLD STANDARD' (NG-STAR) ===
    
    print(f"Step 1/5: Searching for NG-STAR files in directory '{NGSTAR_INPUT_DIR}'...")
    
    # Looking for .xlsx files
    ngstar_files = glob.glob(os.path.join(NGSTAR_INPUT_DIR, "*.xlsx"))
    
    if not ngstar_files:
        print(f"ERROR: No .xlsx files found in '{NGSTAR_INPUT_DIR}'.")
        print("Please create the 'ngstar_outputs' folder and place the 29 XLSX files inside.")
        sys.exit(1)

    print(f"Found {len(ngstar_files)} files. Processing...")
    
    all_ngstar_data = []
    for f in ngstar_files:
        try:
            # Extract strain name from filename
            basename = os.path.basename(f)
            strain_name = basename.split('.')[0] 
            
            # Read Excel file
            df_temp = pd.read_excel(f)
            
            df_temp['Strain'] = strain_name
            
            # Clean locus names
            df_temp['NGSTAR_locus'] = df_temp['Locus'].apply(clean_ngstar_locus)
            
            # Select only relevant columns
            df_clean = df_temp[['Strain', 'NGSTAR_locus', 'Allele', 'Comments']]
            all_ngstar_data.append(df_clean)
            
        except Exception as e:
            print(f"Warning: Failed to process file {f}. Error: {e}")
            # Ensure 'openpyxl' is installed (`pip install openpyxl`)

    # Concatenate all 29 files into one DataFrame
    ngstar_long_master = pd.concat(all_ngstar_data, ignore_index=True)
    
    ngstar_long_master['Allele'] = ngstar_long_master['Allele'].astype(str).replace('100', 'WT/None')
    ngstar_long_master.rename(columns={'Allele': 'NGSTAR_Allele', 'Comments': 'NGSTAR_Comments'}, inplace=True)

    print("Step 1.5: Saving the 'long format' NG-STAR table...")
    ngstar_long_master.to_csv("NGSTAR_master_table_long_format.csv", index=False, encoding='utf-8-sig')

    print("Step 1 complete. New NG-STAR 'Gold Standard' assembled.")

    # === PART 2: PROCESSING TEST DATA (RGI/CARD) ===

    print(f"Step 2/5: Loading and processing RGI data from '{RGI_INPUT_FILE}'...")
    try:
        df_rgi = pd.read_csv(RGI_INPUT_FILE)
    except FileNotFoundError:
        print(f"ERROR: File '{RGI_INPUT_FILE}' not found.")
        print("Please place 'RGI_tidy_filtered.csv' in the same folder as the script.")
        sys.exit(1)

    # Normalize strain names
    df_rgi['Strain'] = df_rgi['Source'].str.replace('_2024', '', regex=False)
    
    # V5 FIX: Applying our debugged mapping logic
    df_rgi['NGSTAR_locus_mapped'] = df_rgi['Best_Hit_ARO'].apply(map_aro_to_ngstar_v5)
    
    # Aggregate RGI data
    agg_functions = {
        'Best_Hit_ARO': aggregate_strings,
        'SNPs_in_Best_Hit_ARO': aggregate_strings,
        'Model_type': aggregate_strings,
        'Best_Identities': 'max',
    }
    
    rgi_summary = df_rgi.groupby(['Strain', 'NGSTAR_locus_mapped']).agg(agg_functions).reset_index()
    rgi_summary.rename(columns={'NGSTAR_locus_mapped': 'NGSTAR_locus'}, inplace=True)
    rgi_summary['SNPs_in_Best_Hit_ARO'].fillna('N/A (Not SNP Model)', inplace=True)

    print("Step 2 complete. RGI data processed.")

    # === PART 3: FINAL MERGE AND ANALYSIS ===

    print("Step 3/5: Merging NG-STAR and RGI tables...")
    
    comparison_df = pd.merge(
        ngstar_long_master,  
        rgi_summary,         
        on=['Strain', 'NGSTAR_locus'],
        how='outer'
    )

    print("Step 4/5: Calculating Concordance...")
    
    comparison_df['Concordance'] = comparison_df.apply(check_concordance, axis=1)
    
    comparison_df = comparison_df[comparison_df['NGSTAR_locus'] != 'Other']
    
    comparison_df.dropna(subset=['NGSTAR_Allele', 'Best_Hit_ARO'], how='all', inplace=True)
    
    comparison_df.sort_values(by=['Strain', 'NGSTAR_locus'], inplace=True)
    
    print("Step 5/5: Saving the final report...")
    
    comparison_df.to_csv(FINAL_OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print("\n" + "="*50)
    print("🚀 SCRIPT EXECUTED SUCCESSFULLY!")
    print(f"Final comparison table saved to:")
    print(f"{FINAL_OUTPUT_FILE}")
    print("="*50)
    print("\n--- Example of final data (WHO_H, if found): ---")
    print(comparison_df[comparison_df['Strain'] == 'WHO_H'])
    print("\n--- Example of 'RGI Only' findings (rpsJ, tetM): ---")
    print(comparison_df[comparison_df['NGSTAR_locus'].isin(['rpsJ', 'tet(M)'])].head())

# Run the script
if __name__ == "__main__":
    main()